In [7]:
import pandas as pd
import numpy as np

In [8]:
DATA_PATH = "../data/"
MED_FILE = "PRESCRIPTIONS.csv"
DIAGS_FILE = "DIAGNOSES_ICD.csv"
NDC_TO_ADC_FILE = "package_NDC_ATC4_classes.csv"

In [9]:
def process_med():
    Field = ["SUBJECT_ID","HADM_ID","ICUSTAY_ID","STARTDATE","NDC"]
    FieldType = {"SUBJECT_ID": 'Int64',
                "HADM_ID": 'Int64',
                "ICUSTAY_ID": 'Int64',
                "NDC": 'str',
                }
    med_df = pd.read_csv(DATA_PATH+MED_FILE, usecols=Field, dtype=FieldType, parse_dates=["STARTDATE"])

    med_df = med_df[med_df['NDC'] != '0'] 
    med_df.fillna(method='pad', inplace=True)
    med_df.dropna(inplace=True) 
    med_df.drop_duplicates(inplace=True)
    med_df.sort_values(by=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTDATE'], inplace=True)
    med_df = med_df.reset_index(drop=True)

    def filter_first24hour_med(med_df):
        med_df_new = med_df.drop(columns=['NDC'])
        med_df_new = med_df_new.groupby(by=['SUBJECT_ID','HADM_ID','ICUSTAY_ID']).head(1).reset_index(drop=True) #returns the first startdate
        med_df_new = pd.merge(med_df_new, med_df, on=['SUBJECT_ID','HADM_ID','ICUSTAY_ID','STARTDATE'])
        med_df_new = med_df_new.drop(columns=['STARTDATE'])
        return med_df_new
    med_df = filter_first24hour_med(med_df) 
    med_df = med_df.drop(columns=['ICUSTAY_ID'])
    med_df = med_df.drop_duplicates()

    return med_df.reset_index(drop=True)

In [10]:
def process_ndc2atc4():
    """ process_ndc2atc4 reformats ndc from 10 digits to 11 digits 
            https://www.michigan.gov/-/media/Project/Websites/lara/healthsystemslicensing/Folder4/lara_MAPS_NDC_Guidelines.pdf?rev=4cb0e4c8d98946659f47dce703dabc22

        return: 2 column array ['NDC','ATC_class']
    """
    Field = ["NDC","ATC_class"]
    FieldType = {"NDC": 'str',
                "ATC_class": 'str'
                }
    ndc2atc_df = pd.read_csv(DATA_PATH+NDC_TO_ADC_FILE, usecols=Field, dtype=FieldType)
    ndc2atc_df[['Seg1', 'Seg2', 'Seg3']] = ndc2atc_df['NDC'].str.split('-', expand=True)
    ndc2atc_df['Seg1'] = ndc2atc_df['Seg1'].str.pad(side="left",width=5,fillchar='0')
    ndc2atc_df['Seg2'] = ndc2atc_df['Seg2'].str.pad(side="left",width=4,fillchar='0')
    ndc2atc_df['Seg3'] = ndc2atc_df['Seg3'].str.pad(side="left",width=2,fillchar='0')
    ndc2atc_df['NDC'] = ndc2atc_df[['Seg1', 'Seg2', 'Seg3']].agg(''.join, axis=1)
    ndc2atc_df = ndc2atc_df.drop_duplicates()
    return ndc2atc_df.filter(items=['NDC','ATC_class'])

In [11]:
med_df = process_med()
ndc2atc_df = process_ndc2atc4()


C:\Users\jakeh\AppData\Local\Temp\ipykernel_10604\3121096391.py:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  med_df.fillna(method='pad', inplace=True)


In [12]:
ndc2atc_df.shape

(337186, 2)

In [13]:
# create map
med_df['NDC'].unique()

np.savetxt(str(DATA_PATH)+"input.txt", med_df['NDC'].unique(), fmt='%s')


In [14]:
med_df.dtypes

SUBJECT_ID     Int64
HADM_ID        Int64
NDC           object
dtype: object

In [15]:
med_df['NDC'].unique()

array(['63323017302', '63323038810', '00088222033', ..., '00904125061',
       '11980002205', '00075800180'], dtype=object)

In [16]:
test = med_df.merge(ndc2atc_df, right_on='NDC',left_on='NDC', how='left')
test = test.drop_duplicates()
test.shape

(1898277, 4)

In [28]:
def process_diag():
    diag_df = pd.read_csv(DATA_PATH+DIAGS_FILE)
    diag_df = diag_df.dropna()
    diag_df = diag_df.drop(columns=['SEQ_NUM','ROW_ID'])
    diag_df = diag_df.drop_duplicates()
    diag_df = diag_df.sort_values(by=['SUBJECT_ID', 'HADM_ID']).reset_index(drop=True)
    return diag_df

In [ ]:
# I did num = 129 instead of 128 to make the shapes match the output of their code since our
# filtering methods are different.

def filter_diag(diag_df, num=129):
    most_common_codes = diag_df['ICD9_CODE'].value_counts().head(num).index
    print(len(most_common_codes))
    diag_df = diag_df[diag_df['ICD9_CODE'].isin(most_common_codes)]
    return diag_df

In [35]:
diag_df = process_diag()
print(diag_df.shape)

diag_df = filter_diag(diag_df)
print(diag_df.shape)

process_diag
(650940, 3)
129
(350136, 3)
